In [1]:
import pandas as pd
from grid_search_utils import get_grid_search_results
import itertools

In [2]:
# give directory name
directory = "02-04-2024"
get_grid_search_results(directory, n_lines=128, header_in_line=1)

    attributes  values  context_unaware  game_size  vocab_size_factor  \
0          3.0     4.0              0.0       10.0                3.0   
1          3.0     4.0              1.0       10.0                3.0   
2          3.0     4.0              1.0       10.0                3.0   
3          3.0     4.0              1.0       10.0                3.0   
4          3.0     4.0              1.0       10.0                3.0   
..         ...     ...              ...        ...                ...   
85         3.0     4.0              1.0       10.0                3.0   
86         3.0    16.0              0.0       10.0                3.0   
87         3.0    16.0              0.0       10.0                3.0   
88         3.0     4.0              1.0       10.0                3.0   
89         3.0     4.0              1.0       10.0                3.0   

    n_epochs  batch_size  learning_rate  hidden_size  temp_update  \
0       60.0        32.0          0.001        128.0  

In [3]:
data = pd.read_csv("results_" + directory + ".csv")

## New grid search results

In [4]:
# parameters that were tested
for col in data.columns[:-6]:
    print(col, data[col].unique())    

attributes [3 5]
values [ 4 16]
context_unaware [0. 1.]
game_size [10]
vocab_size_factor [3.]
n_epochs [60.]
batch_size [32]
learning_rate [0.001]
hidden_size [128]
temp_update [0.99]
temperature [2.]
length_cost [0.0005 0.005  0.001 ]
max_mess_len [30. 20.  3. 10.  5.]


In [5]:
data = data[((data['attributes'] == 3) & (data['values'] == 4))
            | ((data['attributes'] == 3) & (data['values'] == 16))
            | ((data['attributes'] == 5) & (data['values'] == 4))]

In [6]:
attributes = [3, 5]
values = [4, 16]
datasets = list(itertools.product(attributes, values))[:3]
index = data.index
for (attributes, values) in datasets:
    for context_unaware in range(2):
        print("dataset:", attributes, values, "context_unaware:", context_unaware)
        per_dataset = data[(data["attributes"] == attributes) & (data["values"] == values) & (data['context_unaware'] == context_unaware)]
        max_test_acc = per_dataset['test_accuracy'].idxmax()
        print(per_dataset.loc[index[max_test_acc]])
        min_train_loss = per_dataset['train_loss'].idxmin()
        print(per_dataset.loc[index[min_train_loss]])

dataset: 3 4 context_unaware: 0
attributes             3.000000
values                 4.000000
context_unaware        0.000000
game_size             10.000000
vocab_size_factor      3.000000
n_epochs              60.000000
batch_size            32.000000
learning_rate          0.001000
hidden_size          128.000000
temp_update            0.990000
temperature            2.000000
length_cost            0.000500
max_mess_len          10.000000
train_loss             0.011059
train_accuracy         0.999994
train_length          10.998214
test_loss              0.023236
test_accuracy          0.996354
test_length           11.000000
Name: 6, dtype: float64
attributes             3.000000
values                 4.000000
context_unaware        0.000000
game_size             10.000000
vocab_size_factor      3.000000
n_epochs              60.000000
batch_size            32.000000
learning_rate          0.001000
hidden_size          128.000000
temp_update            0.990000
temperature     

### Length cost parameter & maximum message length
Best minimum losses are usually found with low cost parameters (where the cost doesn't influence the training sufficiently). So, we choose costs where it is evident from the message length observed in the last epoch (60) that the cost actually had an influence.
This is the case for 
- 0.005 for D(3,4)
- 0.001 for D(3,16)
- 0.005 for D(5,4)

In [7]:
lc_sumstat = data.groupby(['attributes', 'values', 'context_unaware', 'length_cost', 'max_mess_len'])['test_accuracy', 'test_loss', 'test_length'].describe()
lc_acc = lc_sumstat.loc[:,[('test_accuracy', 'mean')]]
lc_loss = lc_sumstat.loc[:,[('test_loss', 'mean')]]
lc_length = lc_sumstat.loc[:,[('test_length', 'mean')]]
lc_merged = lc_acc.merge(lc_loss, left_index=True, right_index=True)
lc_merged = lc_merged.merge(lc_length, left_index=True, right_index=True)
lc_merged

/var/folders/k6/03rlh8jd6nqbws0_xg9jh20w0000gq/T/ipykernel_8135/2752776918.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  lc_sumstat = data.groupby(['attributes', 'values', 'context_unaware', 'length_cost', 'max_mess_len'])['test_accuracy', 'test_loss', 'test_length'].describe()


test_accuracy  \
                                                                    mean   
attributes values context_unaware length_cost max_mess_len                 
3          4      0.0             0.0005      3.0               0.989227   
                                              5.0               0.994358   
                                              10.0              0.996354   
                                              20.0              0.986211   
                                              30.0              0.986192   
...                                                                  ...   
5          4      1.0             0.0050      3.0               0.896727   
                                              5.0               0.941478   
                                              10.0              0.995088   
                                              20.0              0.968116   
                                              30.0              0.916183   

                                                           test_loss  \
                                                                mean   
attributes values context_unaware length_cost max_mess_len             
3          4      0.0             0.0005      3.0           0.041587   
                                              5.0           0.028957   
                                              10.0          0.023236   
                                              20.0          0.051317   
                                              30.0          0.049808   
...                                                              ...   
5          4      1.0             0.0050      3.0           0.325546   
                                              5.0           0.204935   
                                              10.0          0.054159   
                                              20.0          0.142014   
                                              30.0          0.299898   

                                                           test_length  
                                                                  mean  
attributes values context_unaware length_cost max_mess_len              
3          4      0.0             0.0005      3.0             4.000000  
                                              5.0             6.000000  
                                              10.0           11.000000  
                                              20.0            8.805555  
                                              30.0           11.590278  
...                                                                ...  
5          4      1.0             0.0050      3.0             3.711757  
                                              5.0             3.679388  
                                              10.0            5.826122  
                                              20.0            4.623525  
                                              30.0            4.255035  

[90 rows x 3 columns]

### Length cost

In [8]:
lc_sumstat = data.groupby(['attributes', 'values', 'context_unaware', 'length_cost'])['test_accuracy', 'test_loss', 'test_length'].describe()
lc_acc = lc_sumstat.loc[:,[('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
lc_loss = lc_sumstat.loc[:,[('test_loss', 'mean'), ('test_loss', 'std')]]
lc_length = lc_sumstat.loc[:,[('test_length', 'mean'), ('test_length', 'std')]]
lc_merged = lc_acc.merge(lc_loss, left_index=True, right_index=True)
lc_merged = lc_merged.merge(lc_length, left_index=True, right_index=True)
lc_merged

/var/folders/k6/03rlh8jd6nqbws0_xg9jh20w0000gq/T/ipykernel_80914/1025698828.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  lc_sumstat = data.groupby(['attributes', 'values', 'context_unaware', 'length_cost'])['test_accuracy', 'test_loss', 'test_length'].describe()


test_accuracy            \
                                                       mean       std   
attributes values context_unaware length_cost                           
3          4      0.0             0.0005           0.990468  0.004683   
                                  0.0010           0.983805  0.002454   
                                  0.0050           0.980172  0.006586   
                  1.0             0.0005           0.988861  0.004997   
                                  0.0010           0.991952  0.004920   
                                  0.0050           0.979001  0.007181   
           16     0.0             0.0005           0.971928  0.038506   
                                  0.0010           0.984078  0.007849   
                                  0.0050           0.799340  0.235369   
                  1.0             0.0005           0.983590  0.017600   
                                  0.0010           0.991446  0.003758   
                                  0.0050           0.799849  0.238869   
5          4      0.0             0.0005           0.922830  0.069450   
                                  0.0010           0.950084  0.063422   
                                  0.0050           0.953268  0.041514   
                  1.0             0.0005           0.909666  0.052453   
                                  0.0010           0.984555  0.029604   
                                  0.0050           0.943518  0.039387   

                                              test_loss           test_length  \
                                                   mean       std        mean   
attributes values context_unaware length_cost                                   
3          4      0.0             0.0005       0.038981  0.012496    8.279167   
                                  0.0010       0.061285  0.008787    6.448611   
                                  0.0050       0.086770  0.016116    4.104514   
                  1.0             0.0005       0.048247  0.018218    7.825000   
                                  0.0010       0.042618  0.022503    6.605903   
                                  0.0050       0.089251  0.019386    4.057986   
           16     0.0             0.0005       0.096388  0.108649    9.744303   
                                  0.0010       0.073424  0.027594    4.987907   
                                  0.0050       0.346195  0.308162    2.341946   
                  1.0             0.0005       0.063665  0.048786    9.639876   
                                  0.0010       0.045001  0.015169    5.476502   
                                  0.0050       0.345183  0.312344    2.351967   
5          4      0.0             0.0005       0.272080  0.295129   14.595836   
                                  0.0010       0.158703  0.170447    9.589360   
                                  0.0050       0.178161  0.112672    4.522280   
                  1.0             0.0005       0.241196  0.120152   14.600000   
                                  0.0010       0.068422  0.108259    8.532218   
                                  0.0050       0.205311  0.112090    4.419165   

                                                          
                                                     std  
attributes values context_unaware length_cost             
3          4      0.0             0.0005        3.246126  
                                  0.0010        1.877062  
                                  0.0050        0.279776  
                  1.0             0.0005        2.813684  
                                  0.0010        1.633601  
                                  0.0050        0.107236  
           16     0.0             0.0005        6.964102  
                                  0.0010        0.877896  
                                  0.0050        1.252004  
                  1.0             0.0005        6.903151  
                                  0.0010   

### Maximum message length
No large differences between max_mess_len = 10, 20 or 30 for all datasets. Go with lowest or highest, best or best for all?
Max_mess_len = 20 seems to work well for all.

In [9]:
lc_sumstat = data.groupby(['attributes', 'values', 'context_unaware', 'max_mess_len'])['test_accuracy', 'test_loss', 'test_length'].describe()
lc_acc = lc_sumstat.loc[:,[('test_accuracy', 'mean'), ('test_accuracy', 'std')]]
lc_loss = lc_sumstat.loc[:,[('test_loss', 'mean'), ('test_loss', 'std')]]
lc_length = lc_sumstat.loc[:,[('test_length', 'mean'), ('test_length', 'std')]]
lc_merged = lc_acc.merge(lc_loss, left_index=True, right_index=True)
lc_merged = lc_merged.merge(lc_length, left_index=True, right_index=True)
lc_merged

/var/folders/k6/03rlh8jd6nqbws0_xg9jh20w0000gq/T/ipykernel_80914/319962527.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  lc_sumstat = data.groupby(['attributes', 'values', 'context_unaware', 'max_mess_len'])['test_accuracy', 'test_loss', 'test_length'].describe()


test_accuracy            \
                                                        mean       std   
attributes values context_unaware max_mess_len                           
3          4      0.0             3.0               0.980592  0.010208   
                                  5.0               0.988068  0.005886   
                                  10.0              0.985352  0.009542   
                                  20.0              0.984591  0.001654   
                                  30.0              0.985472  0.001616   
                  1.0             3.0               0.984838  0.002540   
                                  5.0               0.984042  0.010427   
                                  10.0              0.983671  0.010884   
                                  20.0              0.990169  0.009919   
                                  30.0              0.990302  0.005587   
           16     0.0             3.0               0.840766  0.260015   
                                  5.0               0.835097  0.253069   
                                  10.0              0.975795  0.019695   
                                  20.0              0.957110  0.045349   
                                  30.0              0.983475  0.013580   
                  1.0             3.0               0.840331  0.259126   
                                  5.0               0.840421  0.264164   
                                  10.0              0.983840  0.007969   
                                  20.0              0.970804  0.021377   
                                  30.0              0.989411  0.006704   
5          4      0.0             3.0               0.897400  0.056682   
                                  5.0               0.993311  0.001497   
                                  10.0              0.962181  0.057849   
                                  20.0              0.915891  0.071101   
                                  30.0              0.941520  0.048519   
                  1.0             3.0               0.918880  0.019259   
                                  5.0               0.977160  0.030939   
                                  10.0              0.953347  0.075543   
                                  20.0              0.948685  0.060345   
                                  30.0              0.931493  0.061210   

                                               test_loss            \
                                                    mean       std   
attributes values context_unaware max_mess_len                       
3          4      0.0             3.0           0.071679  0.030649   
                                  5.0           0.056630  0.030366   
                                  10.0          0.064605  0.039317   
                                  20.0          0.058824  0.011689   
                                  30.0          0.059990  0.009321   
                  1.0             3.0           0.070660  0.008574   
                                  5.0           0.071246  0.035114   
                                  10.0          0.064441  0.036643   
                                  20.0          0.045113  0.038741   
                                  30.0          0.048733  0.024604   
           16     0.0             3.0           0.256851  0.366852   
                                  5.0           0.283669  0.348271   
                                  10.0          0.094807  0.064535   
                                  20.0          0.150165  0.114673   
                                  30.0          0.074520  0.053896   
                  1.0             3.0           0.260890  0.368850   
                                  5.0           0.255117  0.374328   
                                  10.0          0.076828  0.035735   
                                  20.0          0.107230  0.064341   
                                  30.0          0.056351  0.0351